In [ ]:
!pip install kserve

In [2]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1ModelSpec
from kserve import V1beta1ModelFormat

In [3]:
namespace = utils.get_default_target_namespace()

In [4]:
name='mnist'
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                                 model=V1beta1ModelSpec(
                                     storage_uri='https://huggingface.co/tarilabs/mnist/resolve/v20231206163028/mnist.onnx?download=true',
                                     model_format=V1beta1ModelFormat(name='onnx', version='1')
                                 )
                               )))

In [ ]:
KServe = KServeClient()
KServe.create(isvc)

In [ ]:
KServe.get(name, namespace=namespace, watch=True, timeout_seconds=120)